In [ ]:
!python -m pip install hummingbird_ml

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from hummingbird.ml import convert


In [2]:
# We are going to use the breast cancer dataset from scikit-learn for this example.
X, y = load_breast_cancer(return_X_y=True)
nrows=15000
X = X[0:nrows]
y = y[0:nrows]

In [3]:
# Create and train a random forest model.
model = RandomForestClassifier(n_estimators=10, max_depth=10)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [4]:
%%timeit -r 3

# Time for scikit-learn.
model.predict(X)

1.07 ms ± 14.2 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


In [5]:
model = convert(model, 'torch', extra_config={"tree_implementation":"gemm"})

In [6]:
%%timeit -r 3

# Time for HB.
model.predict(X)

444 µs ± 16.9 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


In [7]:
model.to('cuda')

PyTorchBackendModelClassification(
  (operator_map): ModuleDict(
    (SklearnRandomForestClassifier): GEMMDecisionTreeImpl()
  )
)

In [8]:
%%timeit -r 3

# Time for HB GPU.
model.predict(X)

291 µs ± 16 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)
